In [1]:
!wget https://github.com/UniversalDependencies/UD_Thai-PUD/raw/master/th_pud-ud-test.conllu

--2023-11-06 17:56:02--  https://github.com/UniversalDependencies/UD_Thai-PUD/raw/master/th_pud-ud-test.conllu
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/UniversalDependencies/UD_Thai-PUD/master/th_pud-ud-test.conllu [following]
--2023-11-06 17:56:04--  https://raw.githubusercontent.com/UniversalDependencies/UD_Thai-PUD/master/th_pud-ud-test.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2148691 (2.0M) [text/plain]
Saving to: ‘th_pud-ud-test.conllu’

th_pud-ud-test.conl 100%[===================>]   2.05M  3.76MB/s    in 0.5s    

2023-11-06 17:56:05 (3.76 MB/s) - ‘th_pud-ud-test

In [2]:
import pandas as pd
from pathlib import Path
import os

In [3]:
path = "th_pud-ud-test.conllu"

In [4]:
def readfile(path):
    _data = []
    with open(path,'r',encoding="utf-8-sig") as f:
        _temp = [i for i in f.read().split("\n\n") if i != '']
    for j in _temp:
        _temp_data = []
        bad_data = False
        for i in j.split("\n"):
            if i.startswith('#') == False and not bad_data:
                _t = i.strip().split("\t")
                if _t[1]!="$$":
                    if _t[3]=="N":
                        _temp_data=[]
                        bad_data=True
                    else:
                        _temp_data.append((_t[1],_t[3]))
        if _temp_data!=[]:
            _data.append(_temp_data)
    return _data

In [9]:
train_data_ud = readfile(path)

In [7]:
from collections import defaultdict

In [8]:
pos_dict = defaultdict(list)

In [10]:
for i in train_data_ud:
    _word= [k for k,_ in i]
    _pos = '/'.join([k for _,k in i])
    pos_dict[_pos].append(_word)

In [11]:
temp="จงสร้างประโยคตามโครงสร้าง {pos}:"

In [12]:
inputs=[]
targets=[]

In [13]:
for i in sorted(list(pos_dict.keys())):
    if i.count("/")<1:
        continue
    for k in pos_dict[i]:
        inputs.append(temp.format(pos=i))
        targets.append(''.join(k))

In [14]:
inputs[-1],targets[-1]

('จงสร้างประโยคตามโครงสร้าง VERB/VERB/VERB/VERB/VERB/ADJ/ADP/NOUN/NOUN/DET/VERB/ADP/NOUN/NOUN/CCONJ/NOUN/VERB/NOUN/ADP/NOUN/ADP/NOUN/VERB:',
 'การวิเคราะห์แสดงให้เห็นความสัมพันธ์โดยตรงระหว่างจำนวนบุหรี่ที่สูบตลอดช่วงชีวิตกับจำนวนการกลายพันธุ์ในดีเอ็นเอของเนื้องอก')

In [15]:
df_test = {
    "inputs":inputs,
    "targets":targets,
}

In [17]:
id=0
print(df_test["inputs"][id])
print(df_test["targets"][id])

จงสร้างประโยคตามโครงสร้าง ADJ/ADP/NOUN/ADP/AUX/NOUN/ADP/NOUN/NOUN/CCONJ/NOUN/NOUN/NOUN/NOUN/NOUN/NOUN/CCONJ/NOUN/NOUN/ADJ:
อุดมด้วยแร่ธาตุโดยเป็นแหล่งของทองคำเพชรและสินแร่แมงกานีสตะกั่วสังกะสีนิกเกิลโคบอลต์และโมลิบดีนัมขนาดใหญ่


In [18]:
from datasets import Dataset, DatasetDict

In [19]:
ds = DatasetDict()

In [20]:
ds['test'] = Dataset.from_dict(df_test)

In [21]:
ds.push_to_hub("pythainlp/UD_Thai-PUD-prompt",private=False)

Pushing split test to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]